In [5]:
print('\nload dataset loaded')
import torchvision.transforms as transforms
import torch
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, random_split
def load_datasets(ood, NUM_CLIENTS,BATCH_SIZE):
    print('\nload dataset loaded')
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    if not ood:
        testset = CIFAR10("./dataset", train=False, download=True, transform=transform)
    else:
        transform_ood = transforms.Compose([transforms.ToTensor(), transforms.Normalize((1.0, 0.25, 0.1),(0.5, 0.5, 0.5))])
        testset = CIFAR10("./dataset", train=False, download=True, transform=transform_ood)

    # Split training set into 10 partitions to simulate the individual dataset
    partition_size = len(trainset) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader


load dataset loaded
